In [ ]:
import ast
from collections import Counter
import math
import zlib
import pandas as pd
import numpy as np 
import mido

In [2]:
import os
import pandas as pd
from mido import MidiFile

# Fonction pour extraire les notes au format (p, d, v, t) d'un fichier MIDI
def extract_note_data(midi_path):
    note_data = []
    midi = MidiFile(midi_path)
    current_time = 0  # Temps accumulé en ticks
    durations = {}  # Stocker le temps de début des notes pour calculer la durée

    for track in midi.tracks:
        for msg in track:
            if not msg.is_meta:
                current_time += msg.time  # Incrémenter le temps
                if msg.type == 'note_on' and msg.velocity > 0:  # Note On
                    durations[msg.note] = current_time  # Stocker le temps de début
                    note_data.append((msg.note, 0, msg.velocity, current_time))
                elif msg.type == 'note_off':  # Note Off
                    if msg.note in durations:
                        duration = current_time - durations[msg.note]
                        # Mettre à jour la dernière note pour inclure la durée
                        note_data[-1] = (msg.note, duration, note_data[-1][2], note_data[-1][3])
    return note_data

# Fonction pour parcourir les fichiers et créer un DataFrame
def create_csv_from_midi(root_folder, output_csv):
    data = []  # Liste pour stocker les informations
    for root, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.mid') or file.endswith('.midi'):
                file_path = os.path.join(root, file)
                genre = os.path.basename(os.path.dirname(file_path))  # Récupérer le nom du sous-dossier
                try:
                    # Extraire les notes du fichier MIDI
                    notes = extract_note_data(file_path)
                    note_representation = ";".join([f"({p},{d},{v},{t})" for p, d, v, t in notes])
                    data.append((file, genre, note_representation))
                except Exception as e:
                    print(f"Erreur pour le fichier {file}: {e}")
    
    # Créer un DataFrame
    df = pd.DataFrame(data, columns=['name', 'genre', 'note'])
    # Sauvegarder au format CSV
    df.to_csv(output_csv, index=False)
    print(f"CSV créé : {output_csv}")

# Chemin du dossier contenant les fichiers MIDI
midi_folder = "midi_files_cleaned"  # Remplacez par votre chemin
output_csv = "midi_notes.csv"

# Exécuter la fonction pour créer le CSV
create_csv_from_midi(midi_folder, output_csv)


Erreur pour le fichier Theme from C K467.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Sleepers, Wake.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Nutcracker Minature Overture.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Saltarelle.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier English Suite 3 Gavotte.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Carry That Weight.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Divenire.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Feel This Moment.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Impromptu in Ges-dur.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Clarinet Concerto Theme.mid: Could not decode key with 8 sharps and mode 0
Erreur pour le fichier Arthurs Theme.mid: Could not decode key with 8 shar

KeyboardInterrupt: 

In [3]:
print(extract_note_data("midi_files_cleaned/ -start ID-.mid"))

[(36, 0, 108, 1616), (60, 0, 104, 1616), (63, 0, 98, 1616), (72, 0, 101, 1616), (24, 0, 107, 1618), (67, 0, 101, 1620), (55, 0, 51, 1988), (60, 0, 41, 2166), (72, 0, 108, 3892), (63, 0, 101, 3894), (75, 0, 112, 4196), (67, 0, 112, 4198), (79, 0, 101, 4482), (72, 0, 106, 4484), (24, 0, 112, 4666), (36, 0, 106, 4668), (69, 0, 112, 4668), (77, 0, 114, 4668), (65, 0, 45, 4864), (57, 0, 92, 5068), (65, 0, 90, 5076), (53, 0, 59, 5260), (60, 0, 79, 5358), (60, 0, 98, 6080), (53, 0, 92, 6082), (55, 0, 90, 6144), (60, 0, 99, 6216), (55, 0, 97, 6218), (57, 0, 99, 6260), (48, 0, 73, 7258), (53, 0, 121, 7356), (60, 0, 116, 7356), (55, 0, 107, 7440), (48, 0, 108, 7538), (51, 0, 107, 7738), (24, 0, 112, 7740), (36, 0, 110, 7740), (79, 0, 114, 8126), (67, 0, 114, 8128), (74, 0, 116, 8130), (75, 0, 114, 8130), (72, 0, 92, 8314), (62, 0, 84, 9214), (67, 0, 51, 9232), (63, 0, 87, 9248), (48, 0, 91, 9272), (62, 0, 91, 9272), (58, 0, 92, 9276), (55, 0, 99, 9278), (58, 0, 108, 10720), (67, 0, 108, 10720), 

In [12]:
import os
import pandas as pd
from mido import MidiFile

# Fonction pour extraire les notes avec (p, d, v, t)
def extract_note_data(midi_path):
    note_data = []
    midi = MidiFile(midi_path)
    current_time = 0  # Temps accumulé en ticks
    active_notes = {}  # Dictionnaire pour stocker les notes et leurs débuts

    for track in midi.tracks:
        for msg in track:
            if not msg.is_meta:
                current_time += msg.time  # Incrémenter le temps en ticks
                if msg.type == 'note_on' and msg.velocity > 0:  # Note On
                    active_notes[msg.note] = (current_time, msg.velocity)  # Stocker temps début et vélocité
                elif msg.type in ['note_off', 'note_on'] and msg.velocity == 0:  # Note Off
                    if msg.note in active_notes:
                        start_time, velocity = active_notes.pop(msg.note)  # Récupérer début et vélocité
                        duration = current_time - start_time  # Calculer durée
                        note_data.append((msg.note, duration, velocity, start_time))  # Ajouter la note
    return note_data

# Créer le CSV avec name, genre et note
def create_csv_from_midi(midi_folder, output_csv):
    data = []

    for root, _, files in os.walk(midi_folder):
        for file in files:
            if file.endswith('.mid') or file.endswith('.midi'):
                file_path = os.path.join(root, file)
                genre = os.path.basename(os.path.dirname(file_path))  # Le genre est le nom du sous-dossier
                try:
                    notes = extract_note_data(file_path)
                    note_representation = ";".join([f"({p},{d},{v},{t})" for p, d, v, t in notes])
                    data.append((file, genre, note_representation))
                except Exception as e:
                    print(f"Erreur avec {file}: {e}")
                    data.append((file, genre, "ERROR"))
    
    # Créer et sauvegarder le DataFrame
    df = pd.DataFrame(data, columns=['name', 'genre', 'note'])
    df.to_csv(output_csv, index=False)
    print(f"CSV sauvegardé à : {output_csv}")

# Chemin des fichiers MIDI et sortie
midi_folder = 'reduced_midi_files'  # Chemin vers vos fichiers extraits
output_csv = 'midi_notes_fixed.csv'

# Exécution
create_csv_from_midi(midi_folder, output_csv)




CSV sauvegardé à : midi_notes_fixed.csv


In [ ]:
midi_folder = 'reduced_midi_files'  # Chemin où sont les fichiers MIDI
output_csv = 'midi_notes_fixed.csv'

In [21]:
def extract_note_data(midi_path):
    note_data = []  # Liste pour stocker (p, d, v, t)
    current_time = 0  # Temps accumulé en ticks
    active_notes = {}  # Stocker les notes en cours {pitch: (start_time, velocity)}

    # Charger le fichier MIDI
    midi = MidiFile(midi_path)

    # Parcourir tous les messages de la track unique
    for i,msg in enumerate(midi) :
        if (msg.type = meta)
        if(i==24) :
            print(msg)


extract_note_data('reduced_midi_files/ -start ID-.mid')

note_on channel=0 note=60 velocity=41 time=0.47952458333333325


**Format :**

'msg.type' espace 'channel' espace 'p' espace 'v' espace 'delta_t'

--> On s'en fiche de channel on l'enlève

--> msg.type c'est soit on soit off donc 0 ou 1 (1 bit)

--> p sur 7 bits et v sur 7 bits

--> delta_t sur 14 bits du coup

--> Donc en fait quand on a des répétitions de p,v,delta_t ça revient au même qu'avant donc on peut réutiliser nos trucs 

--> Les note_off on s'en fout de v donc on code juste 0 puis note puis le delta_t

--> On va juste rajouter les harmoniques 

On va pas utiliser les noms classiques des accords. On utilise un code préfixe pour créer ptit à petit notre ensemble d'accords. Vu qu'est un code préfixe, on pourra déerminer avec certitude de quel accord il s'agit. 

Pour chaque .midi, on aura donc un dictionnairr avec clé = accord et valeur = encodage. 

In [ ]:
import os
import csv
import hashlib

In [ ]:
import load_data as load